In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
# Load the dataset
data = pd.read_csv('Churn_Modelling.csv') 
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data

### Drop unnecessary columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
## Apply Encoding to categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
## Apply One-Hot Encoding to Geography
ohe_gepgraphy = OneHotEncoder(sparse_output=True)
geo_encoder = ohe_gepgraphy.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
ohe_gepgraphy.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(), columns = ohe_gepgraphy.get_feature_names_out(['Geography']))

In [8]:
## Combine the one hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)

In [10]:
## Divide the data into features and target variable
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the featires (Standard Scaler)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)



In [11]:

## Saving the encoders as pkl files
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_gepgraphy, file)

with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(scalar, file)

## ANN for regression Implementation

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [15]:
## Build ANN
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  ## Hiddel layer 1 Connected with I/P layer
    Dense(32, activation='relu'),  ## Hidden layer 2
    Dense(1)  ## Output layer (Linear Act Func by Default)
]
)

c:\Users\mitta\OneDrive\Desktop\NLP Projects\Churn Prediction using ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
## Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
## Setup tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [22]:
## Setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
### Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test), epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback],
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100636.0703 - mae: 100636.0703 - val_loss: 98461.3125 - val_mae: 98461.3125
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100646.8984 - mae: 100646.8984 - val_loss: 96685.1250 - val_mae: 96685.1250
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98295.6562 - mae: 98295.6562 - val_loss: 92260.5703 - val_mae: 92260.5703
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 92283.6641 - mae: 92283.6641 - val_loss: 85086.2031 - val_mae: 85086.2031
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 84229.0156 - mae: 84229.0156 - val_loss: 76001.0547 - val_mae: 76001.0547
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74621.1719 - mae: 74621.1719 - val_loss: 67014.6719 - val_mae: 67014.6719
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 65241.3711 - mae: 65241.3711 - val_loss: 59556.2383 - val_mae: 59556.2383
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste

In [24]:
## Load TensorBoard extention
%load_ext tensorboard

In [26]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 20248), started 0:00:21 ago. (Use '!kill 20248' to kill it.)

In [27]:
# Evaluate on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51073.7930 - mae: 51073.7930
Test Loss: 50358.50390625, Test MAE: 50358.50390625


In [28]:
model.save('regression_model.h5')